In [2]:
from typing import TypedDict, List, Union
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import AzureChatOpenAI
from langgraph.graph import StateGraph, START, END
import os


openai_endpoint = 'https://learningoai-si.openai.azure.com/'#os.getenv("openai_endpoint", "https://your-resource-name.openai.azure.com/")
openai_apikey = os.getenv("openai_apikey", "{your-api-key}")
#openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview
print(f"Using OpenAI Endpoint: {openai_endpoint}")

class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]
    
    
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    azure_endpoint=openai_endpoint,
    api_version="2025-01-01-preview",
    api_key=openai_apikey)

Using OpenAI Endpoint: https://learningoai-si.openai.azure.com/


In [3]:
def process(state: AgentState) -> AgentState:
    """This node will solve the request you input"""
    response = llm.invoke(state["messages"])
    print(f"\nAI: {response.content}")
    state["messages"].append(AIMessage(content=response.content))
    return state

In [4]:
graph = StateGraph(AgentState)
graph.add_node('process',process)
graph.add_edge(START, 'process')
graph.add_edge('process', END)
agent = graph.compile()

In [10]:
conversation_history = []
user_input = input("User: ")

while user_input.lower() not in ['exit', 'quit']:
    conversation_history.append(HumanMessage(content=user_input))
    result = agent.invoke({"messages": conversation_history})
    user_input = input("User: ")


AI: Hello! How can I assist you today? 😊

AI: Hi Manoj! Nice to meet you. 😊 How can I assist you today?

AI: Your name is Manoj! 😊 How can I help you today?


In [12]:

with open("converstion_log.txt", "w", encoding="utf-8") as file:
    for message in conversation_history:
        if isinstance(message, HumanMessage):
            file.write(f"User: {message.content}\n")
        elif isinstance(message, AIMessage):
            file.write(f"AI: {message.content}\n")
            
